In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [ ]:
supplier = pd.read_excel('important_files/Bids.xlsx',  sheet_name='supplier')
supplier.head()

In [ ]:
supplier.dtypes


In [ ]:
supplier.columns = map(str.lower, supplier.columns)
supplier = supplier.fillna('N/A')
supplier['isbn'] = supplier['isbn'].astype(str)
supplier['isbn'] = supplier.apply(lambda x: clean_up(x['isbn']), axis =1)
supplier = supplier.sort_values(['cost'], ascending=True).reset_index(drop=True)
supplier.head()

In [ ]:
customer = pd.read_excel('important_files/Bids.xlsx',  sheet_name='customer')
customer.head()

In [ ]:
customer.columns = map(str.lower, customer.columns)
customer = customer.fillna('N/A')
customer['isbn13'] = customer['isbn13'].astype(str)
for col in ['isbn13', 'customer']:
    customer[col] = customer.apply(lambda x: clean_up(x[col]), axis =1)
customer.rename(columns={'isbn13': 'isbn'}, inplace = True)
customer['cust_qty'] =customer['cust_qty'].round(decimals = 0)
customer.head()

In [ ]:
customer = customer.sort_values(['price'], ascending=False).reset_index(drop=True)
customer.head()

In [ ]:
# bid = pd.DataFrame(columns=['isbn', 'tb cost','warehouse 2', 'qty', 'price', 'customer'])

In [ ]:
bid = pd.DataFrame()
for cust_ind in list(customer.index.values):
    for su_ind in list(supplier.index.values):
        if (supplier.iloc[su_ind, :]['isbn'] == customer.iloc[cust_ind, :]['isbn']) and (supplier.iloc[su_ind, :]['sup_qty'] >= 0) and (customer.iloc[cust_ind, :]['cust_qty'] >= 0) and ((customer.iloc[cust_ind, :]['price'] - supplier.iloc[su_ind, :]['cost']) > -5 ):
            row_bid = pd.merge(supplier.iloc[[su_ind], :], customer.iloc[[cust_ind], :], how='left', on = 'isbn' )
            row_bid['allocate_qty'] = row_bid[['sup_qty','cust_qty']].min( axis = 1, skipna = True)
            row_bid['unit_diff'] = (customer.iloc[cust_ind, :]['price'] - supplier.iloc[su_ind, :]['cost'])
            supplier.loc[su_ind,'sup_qty'] = supplier.loc[su_ind,'sup_qty'] - row_bid.iloc[0,:]['allocate_qty']
            customer.loc[cust_ind,'cust_qty'] = customer.loc[cust_ind,'cust_qty'] - row_bid.iloc[0,:]['allocate_qty']
            bid = bid.append(row_bid)
# row_bid.head()


In [ ]:
bid.head()

In [ ]:

with pd.ExcelWriter('bid/bid.xlsx') as writer:
    bid.to_excel(writer,sheet_name = 'bid', index= False)
    supplier.to_excel(writer, sheet_name = 'supplier', index= False)
    customer.to_excel(writer, sheet_name = 'customer', index= False)
